In [1]:
import h5py
import numpy as np
import multiprocessing as mp

from tqdm import tqdm
from pathlib import Path
from scipy.stats import skew
from itertools import repeat

N_CORES = 64

In [2]:
def DeltaR(eta1, phi1, eta2, phi2):
    dEta = eta1 - eta2
    dPhi = np.abs(phi1 - phi2)
    dPhi = np.where(dPhi > np.pi, 2 * np.pi - dPhi, dPhi)
    dR = np.sqrt(dPhi**2 + dEta**2)
    return dR


def DeltaA(eta1, phi1, eta2, phi2):
    dEta = eta1 - eta2
    dPhi = np.abs(phi1 - phi2)
    dPhi = np.where(dPhi > np.pi, 2 * np.pi - dPhi, dPhi)
    dA = np.cosh(dEta) - np.cos(dPhi)
    return dA


def PxPyPzE(jets):
    # jets: 一個形狀為 (n, 4) 的 NumPy 陣列，其中 n 是噴射數量，每個噴射有四個屬性（pt, eta, phi, m）
    pt, eta, phi, m = jets.T

    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pz = pt * np.sinh(eta)
    e = np.sqrt(m*m + px*px + py*py + pz*pz)

    return px.sum(), py.sum(), pz.sum(), e.sum()


def PtEtaPhiM(px, py, pz, e):

    P = np.sqrt(px**2 + py**2 + pz**2)
    pt = np.sqrt(px**2 + py**2)
    eta = 1 / 2 * np.log((P + pz) / (P - pz))
    phi = np.arctan(py / px)
    m = np.sqrt(e**2 - px**2 - py**2 - pz**2)

    return pt, eta, phi, m

In [3]:
def construct_inputs(h5_file, start, end):

    with h5py.File(h5_file, 'r') as f:

        dR = [[], [], []]
        rms_dR = []
        dA_skew = []
        HT = []
        mhCostheta = []
        eta_mhhh_fraction = []
        sphericity = []
        aplanarity = []

        label = f['CLASSIFICATIONS/EVENT/signal'][start:end]

        for event in tqdm(range(start, end)):

            nj = f['INPUTS/Source/MASK'][event].sum()
            pt = f['INPUTS/Source/pt'][event]
            eta = f['INPUTS/Source/eta'][event]
            phi = f['INPUTS/Source/phi'][event]
            mass = f['INPUTS/Source/mass'][event]
            btag = f['INPUTS/Source/btag'][event]

            # for pairing
            jets_index = np.where(btag)[0][0:6]

            h1b1 = f['TARGETS/h1/b1'][event]
            h1b2 = f['TARGETS/h1/b2'][event]
            h2b1 = f['TARGETS/h2/b1'][event]
            h2b2 = f['TARGETS/h2/b2'][event]
            h3b1 = f['TARGETS/h3/b1'][event]
            h3b2 = f['TARGETS/h3/b2'][event]

            dR1 = DeltaR(eta[h1b1], phi[h1b1], eta[h1b2], phi[h1b2])
            dR2 = DeltaR(eta[h2b1], phi[h2b1], eta[h2b2], phi[h2b2])
            dR3 = DeltaR(eta[h3b1], phi[h3b1], eta[h3b2], phi[h3b2])

            dR[0].append(dR1)
            dR[1].append(dR2)
            dR[2].append(dR3)

            # compute rms of dR, consider all possible combinations
            dR_dijets = [DeltaR(eta[i], phi[i], eta[j], phi[j]) for i in jets_index for j in jets_index if i < j]
            rms_dR.append(np.sqrt(np.mean(np.square(dR_dijets))))

            # compute skewness of dA
            dA_dijets = [DeltaA(eta[i], phi[i], eta[j], phi[j]) for i in jets_index for j in jets_index if i < j]
            dA_skew.append(skew(dA_dijets))

            jets = np.array([[pt[i], eta[i], phi[i], mass[i]] for i in [h1b1, h1b2, h2b1, h2b2, h3b1, h3b2]])

            _, _, _, mh1 = PtEtaPhiM(*PxPyPzE(jets[[0, 1]]))
            _, _, _, mh2 = PtEtaPhiM(*PxPyPzE(jets[[2, 3]]))
            _, _, _, mh3 = PtEtaPhiM(*PxPyPzE(jets[[4, 5]]))

            HT.append(jets[:, 0].sum())

            mh_ref = np.array([120, 115, 110])
            mh_rec = np.array([mh1, mh2, mh3]) - mh_ref

            mhCostheta.append(mh_rec.dot(mh_ref) / (np.linalg.norm(mh_rec) * np.linalg.norm(mh_ref)))

            # eta - mhhh fraction
            _, _, _, mhhh = PtEtaPhiM(*PxPyPzE(jets))

            tmp = 0
            for i in jets_index:
                for j in jets_index:
                    if i < j:
                        tmp += 2 * pt[i] * pt[j] * (np.cosh(eta[i] - eta[j]) - 1)

            eta_mhhh_fraction.append(tmp / mhhh**2)

            # Sphericity and Aplanarity
            Mxyz = np.zeros((3, 3))
            p_total = 0
            for i in range(6):
                px, py, pz, _ = PxPyPzE(jets[i])

                Mxyz += np.outer([px, py, pz], [px, py, pz])

                p_total += (px**2 + py**2 + pz**2)

            Mxyz /= p_total
            eigvals = np.linalg.eigvals(Mxyz)
            eigvals = np.sort(eigvals)[::-1]

            sphericity.append(3 / 2 * (eigvals[1] + eigvals[2]))
            aplanarity.append(3 / 2 * eigvals[2])

        # save the features to npy file
        results = np.array([dR[0], dR[1], dR[2], rms_dR, dA_skew, HT, mhCostheta, eta_mhhh_fraction, sphericity, aplanarity, label]).transpose()
        
        return results

In [4]:
def from_h5_to_DNN_feature(h5_file, output_file):
    # file_path: input h5 file path
    # output_path: output h5 file path with jet pairing results
    # use_btag: whether to use btag information


    with h5py.File(h5_file, 'r') as f:
        nevent = f['INPUTS/Source/pt'].shape[0]
    print(f'Number of events: {nevent}')

    # Multi-core processing
    print(f'Number of cores: {N_CORES}')
    start = [nevent // N_CORES * i for i in range(N_CORES)]
    end = [nevent // N_CORES * (i+1) for i in range(N_CORES)]
    end[-1] = nevent

    with mp.Pool(processes=N_CORES) as pool:
        results = pool.starmap(construct_inputs, zip(repeat(h5_file), start, end))

    data = np.concatenate(results)
    # save the features and label to npy file
    # the last column is the label
    features, labels = data[:, :-1], data[:, -1]
    np.save(output_file.replace('.npy', '-data.npy'), features)
    np.save(output_file.replace('.npy', '-label.npy'), labels)


In [5]:
h5_file = '../Sample/SPANet/sig/gghhh-4pT40_6b-chi2_pairing.h5'
output_file = './DNN_features/resonant_6b_new-chi2_pairing.npy'
from_h5_to_DNN_feature(h5_file, output_file)

Number of events: 50000
Number of cores: 64


100%|██████████| 781/781 [00:04<00:00, 175.60it/s]

100%|██████████| 781/781 [00:04<00:00, 170.27it/s]

100%|██████████| 781/781 [00:05<00:00, 144.95it/s]

100%|██████████| 781/781 [00:06<00:00, 123.50it/s]


In [5]:
h5_file = '../../SPANet2/data/triHiggs/gghhh_6b_PT40_test-abs_pairing.h5'
output_file = './DNN_features/resonant-abs_pairing.npy'
from_h5_to_DNN_feature(h5_file, output_file)

  0%|          | 0/40000 [00:00<?, ?it/s]

100%|██████████| 40000/40000 [02:30<00:00, 265.58it/s]


In [6]:
h5_file = '../../SPANet2/data/triHiggs/gghhh_6b_PT40_test-4b_SPANet_pairing.h5'
output_file = './DNN_features/resonant-4b_SPANet_pairing.npy'
from_h5_to_DNN_feature(h5_file, output_file)

100%|██████████| 40000/40000 [02:07<00:00, 314.95it/s]


In [7]:
h5_file = '../Sample/SPANet/pp6b_6b-chi2_pairing.h5'
output_file = './DNN_features/pp6b-chi2_pairing.npy'
from_h5_to_DNN_feature(h5_file, output_file)

100%|██████████| 28755/28755 [01:59<00:00, 241.23it/s]


In [4]:
h5_file = '../Sample/SPANet/pp6b_6b-abs_pairing.h5'
output_file = './DNN_features/pp6b-abs_pairing.npy'
from_h5_to_DNN_feature(h5_file, output_file)

100%|██████████| 28755/28755 [02:02<00:00, 234.94it/s]


In [2]:
h5_file = '../Sample/SPANet/gghhh_6b_PT40_DNN-chi2_pairing.h5'
output_file = './DNN_features/resonant-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/gghhh_6b_PT40_DNN-abs_pairing.h5'
output_file = './DNN_features/resonant-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/gghhh_6b_PT40_DNN-4b_SPANet_pairing.h5'
output_file = './DNN_features/resonant-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../../SPANet2/data/triHiggs/gghhh_6b_PT40_3h_test.h5'
output_file = './DNN_features/resonant-truth_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/pp6b_6b-chi2_pairing.h5'
output_file = './DNN_features/pp6b-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/pp6b_6b-abs_pairing.h5'
output_file = './DNN_features/pp6b-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/pp6b_6b-4b_SPANet_pairing.h5'
output_file = './DNN_features/pp6b-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

python from_h5_to_DNN_features.py ../Sample/SPANet/gghhh_6b_PT40_DNN-chi2_pairing.h5 ./DNN_features/resonant-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/gghhh_6b_PT40_DNN-abs_pairing.h5 ./DNN_features/resonant-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/gghhh_6b_PT40_DNN-4b_SPANet_pairing.h5 ./DNN_features/resonant-4b_SPANet_pairing.npy &
python from_h5_to_DNN_features.py ../../SPANet2/data/triHiggs/gghhh_6b_PT40_3h_test.h5 ./DNN_features/resonant-truth_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/pp6b_6b-chi2_pairing.h5 ./DNN_features/pp6b-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/pp6b_6b-abs_pairing.h5 ./DNN_features/pp6b-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/pp6b_6b-4b_SPANet_pairing.h5 ./DNN_features/pp6b-4b_SPANet_pairing.npy &


# 4b samples

In [2]:
h5_file = '../Sample/SPANet/gghhh_4b_PT40_DNN-chi2_pairing.h5'
output_file = './DNN_features/resonant_4b-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/gghhh_4b_PT40_DNN-abs_pairing.h5'
output_file = './DNN_features/resonant_4b-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/gghhh_4b_PT40_DNN-4b_SPANet_pairing.h5'
output_file = './DNN_features/resonant_4b-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/pp6b_4b-chi2_pairing.h5'
output_file = './DNN_features/pp6b_4b-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/pp6b_4b-abs_pairing.h5'
output_file = './DNN_features/pp6b_4b-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/pp6b_4b-4b_SPANet_pairing.h5'
output_file = './DNN_features/pp6b_4b-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

python from_h5_to_DNN_features.py ../Sample/SPANet/gghhh_4b_PT40_DNN-chi2_pairing.h5 ./DNN_features/resonant_4b-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/gghhh_4b_PT40_DNN-abs_pairing.h5 ./DNN_features/resonant_4b-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/gghhh_4b_PT40_DNN-4b_SPANet_pairing.h5 ./DNN_features/resonant_4b-4b_SPANet_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/pp6b_4b-chi2_pairing.h5 ./DNN_features/pp6b_4b-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/pp6b_4b-abs_pairing.h5 ./DNN_features/pp6b_4b-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/pp6b_4b-4b_SPANet_pairing.h5 ./DNN_features/pp6b_4b-4b_SPANet_pairing.npy &


# 1M 4b + 50k 6b

In [6]:
h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-train-chi2_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-train-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-train-abs_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-train-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-train-4b_SPANet_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-train-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-test-chi2_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-test-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-test-abs_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-test-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-test-4b_SPANet_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-test-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-train-chi2_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-train-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-train-abs_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-train-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-train-4b_SPANet_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-train-4b_SPANet_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-test-chi2_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-test-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-test-abs_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-test-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-test-4b_SPANet_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-test-4b_SPANet_pairing.npy &


In [1]:
h5_file = '../Sample/SPANet/triHiggs-4pT40_6b-chi2_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_6b-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_6b-abs_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_6b-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_6b-4b_SPANet_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_6b-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_6b-chi2_pairing.h5 ./DNN_features/triHiggs-4pT40_6b-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_6b-abs_pairing.h5 ./DNN_features/triHiggs-4pT40_6b-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_6b-4b_SPANet_pairing.h5 ./DNN_features/triHiggs-4pT40_6b-4b_SPANet_pairing.npy &


# Mixed mass: 1M 4b + 50k 6b

In [1]:
h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-mix-train-chi2_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-mix-train-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-mix-train-abs_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-mix-train-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-mix-train-4b_SPANet_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-mix-train-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-mix-test-chi2_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-mix-test-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-mix-test-abs_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-mix-test-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_4b-mix-test-4b_SPANet_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_4b-mix-test-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_6b-mix-chi2_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_6b-mix-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_6b-mix-abs_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_6b-mix-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = '../Sample/SPANet/triHiggs-4pT40_6b-mix-4b_SPANet_pairing.h5'
output_file = './DNN_features/triHiggs-4pT40_6b-mix-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-mix-train-chi2_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-mix-train-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-mix-train-abs_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-mix-train-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-mix-train-4b_SPANet_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-mix-train-4b_SPANet_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-mix-test-chi2_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-mix-test-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-mix-test-abs_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-mix-test-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/triHiggs-4pT40_4b-mix-test-4b_SPANet_pairing.h5 ./DNN_features/triHiggs-4pT40_4b-mix-test-4b_SPANet_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPAN

In [3]:
for m3_m2 in ['420_280', '500_275', '500_300', '520_325']:
    h5_dir = Path(f'../Sample/SPANet/sig/gghhh_bsm_{m3_m2}')
    output_dir = Path(f'./DNN_features/sig/gghhh_bsm_{m3_m2}')
    output_dir.mkdir(parents=True, exist_ok=True)

    h5_file = h5_dir / 'gghhh-4pT40_4b-test-chi2_pairing.h5'
    output_file = output_dir / 'gghhh-4pT40_4b-test-chi2_pairing.npy'
    cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
    print(cmd)

    h5_file = h5_dir / 'gghhh-4pT40_4b-test-abs_pairing.h5'
    output_file = output_dir / 'gghhh-4pT40_4b-test-abs_pairing.npy'
    cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
    print(cmd)

    h5_file = h5_dir / 'gghhh-4pT40_4b-test-4b_SPANet_pairing.h5'
    output_file = output_dir / 'gghhh-4pT40_4b-test-4b_SPANet_pairing.npy'
    cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
    print(cmd)

    h5_file = h5_dir / 'gghhh-4pT40_6b-1-chi2_pairing.h5'
    output_file = output_dir / 'gghhh-4pT40_6b-chi2_pairing.npy'
    cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
    print(cmd)

    h5_file = h5_dir / 'gghhh-4pT40_6b-1-abs_pairing.h5'
    output_file = output_dir / 'gghhh-4pT40_6b-abs_pairing.npy'
    cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
    print(cmd)

    h5_file = h5_dir / 'gghhh-4pT40_6b-1-4b_SPANet_pairing.h5'
    output_file = output_dir / 'gghhh-4pT40_6b-4b_SPANet_pairing.npy'
    cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
    print(cmd)

h5_dir = Path(f'../Sample/SPANet/bkg')
output_dir = Path(f'./DNN_features/bkg')
output_dir.mkdir(parents=True, exist_ok=True)

h5_file = h5_dir / 'pp6b-4pT40_4b-test-chi2_pairing.h5'
output_file = output_dir / 'pp6b-4pT40_4b-test-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = h5_dir / 'pp6b-4pT40_4b-test-abs_pairing.h5'
output_file = output_dir / 'pp6b-4pT40_4b-test-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = h5_dir / 'pp6b-4pT40_4b-test-4b_SPANet_pairing.h5'
output_file = output_dir / 'pp6b-4pT40_4b-test-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = h5_dir / 'pp6b-4pT40_6b-1-chi2_pairing.h5'
output_file = output_dir / 'pp6b-4pT40_6b-chi2_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = h5_dir / 'pp6b-4pT40_6b-1-abs_pairing.h5'
output_file = output_dir / 'pp6b-4pT40_6b-abs_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

h5_file = h5_dir / 'pp6b-4pT40_6b-1-4b_SPANet_pairing.h5'
output_file = output_dir / 'pp6b-4pT40_6b-4b_SPANet_pairing.npy'
cmd = f'python from_h5_to_DNN_features.py {h5_file} {output_file} &'
print(cmd)

python from_h5_to_DNN_features.py ../Sample/SPANet/sig/gghhh_bsm_420_280/gghhh-4pT40_4b-test-chi2_pairing.h5 DNN_features/sig/gghhh_bsm_420_280/gghhh-4pT40_4b-test-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/sig/gghhh_bsm_420_280/gghhh-4pT40_4b-test-abs_pairing.h5 DNN_features/sig/gghhh_bsm_420_280/gghhh-4pT40_4b-test-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/sig/gghhh_bsm_420_280/gghhh-4pT40_4b-test-4b_SPANet_pairing.h5 DNN_features/sig/gghhh_bsm_420_280/gghhh-4pT40_4b-test-4b_SPANet_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/sig/gghhh_bsm_420_280/gghhh-4pT40_6b-1-chi2_pairing.h5 DNN_features/sig/gghhh_bsm_420_280/gghhh-4pT40_6b-chi2_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/sig/gghhh_bsm_420_280/gghhh-4pT40_6b-1-abs_pairing.h5 DNN_features/sig/gghhh_bsm_420_280/gghhh-4pT40_6b-abs_pairing.npy &
python from_h5_to_DNN_features.py ../Sample/SPANet/sig/gghhh_bsm_420_280/gghhh-4pT40_6b-1-4b_SP